In [33]:
import pandas as pd


In [34]:
def summarize_dataset(file_path):
    # Read the Parquet file
    df = pd.read_parquet(file_path)
    
    # Calculate missing values per column
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    data_types = df.dtypes  # Get the data types of each column
    
    # Prepare the summary DataFrame
    summary_df = pd.DataFrame({
        'Column': missing_values.index,
        'Missing Values': missing_values.values,
        'Percentage Missing (%)': missing_percentage.values,
        'Data Type': data_types.values  # Add the data types to the summary
    })
    
    # Sort the summary DataFrame by the number of missing values, descending
    summary_df = summary_df.sort_values(by='Missing Values', ascending=False)
    
    # Reset index for neat presentation
    summary_df.reset_index(drop=True, inplace=True)
    
    return summary_df


parquet_files = {
    "aed_locations": r"Data\Data\aed_locations.parquet.gzip",
    "cad9": r"Data\Data\cad9.parquet.gzip",
    "interventions_bxl": r"Data\Data\interventions_bxl.parquet.gzip",
    "ambulance_locations": r"Data\Data\ambulance_locations.parquet.gzip",
    "interventions_bxl": r"Data\Data\interventions_bxl.parquet.gzip",
    "interventions_bxl2": r"Data\Data\interventions_bxl2.parquet.gzip",
    "interventions1": r"Data\Data\interventions1.parquet",
    "interventions2": r"Data\Data\interventions2.parquet",
    "interventions3": r"Data\Data\interventions3.parquet",
    "mug_locations": r"Data\Data\mug_locations.parquet.gzip",
    "pit_locations": r"Data\Data\pit_locations.parquet.gzip"
}
    

#### Drop 
- PostalCode permanence,CityName permanence,StreetName permanence,HouseNumber permanence, -> These are already in the long and lat value + high missing value
- Permanence short name,	Permanence long name -> can be used as identifier but delete at least one if not both 
- EventType Firstcall, EventType Trip -> keep only those realted to heart problems: Chest pain, P039 - Cardiac problem (other than thoracic pain)','P019 - Unconscious - syncope', 'P003 - Cardiac arrest','P038 - Person does not answer the call', 'P008 - Patient with defibrillator - pacemaker'
- Delete every observation that has a 'abondon reason?' as these people were 'fixed' 
- drop everything related to location that is not coordiantes 

Transform T0 -> T9 to a date column for the incident and a time column; this can be used to calculate time to get to person and to see how long it takes for the person to get to the hospital 

All 'intervention' datasets have simmilar structure so i propose the same as above for all. 

In [35]:
def process_interventions_dataset(file_path):
    # List of heart disorder related event types
    heart_disorder_events = [
        'P011 - Chest pain',
        'P039 - Cardiac problem (other than thoracic pain)',
        'P019 - Unconscious - syncope',
        'P003 - Cardiac arrest',
        'P038 - Person does not answer the call',
        'P008 - Patient with defibrillator - pacemaker'
    ]

    # Columns to drop
    columns_to_drop = [
    'postalcode_intervention', 'name_destination_hospital',
    'postalcode_destination_hospital', 'cityname_destination_hospital',
    'streetname_destination_hospital', 'housenumber_destination_hospital',
    'eventtype_firstcall', 'cityname_intervention', 'postalcode_intervention',
    'permanence_long_name', 'postalcode_permanence', 'cityname_permanence',
    'streetname_permanence', 'housenumber_permanence', 'service_name', 'abandon_reason',
    'eventtype_trip', 'unavailable_time', 't9','permanence_short_name','eventlevel_firstcall','eventlevel_trip',
    't1confirmed','intervention_time_(t1reported)','departure_time_(t1reported)'
]

    
    # Load the dataset for parquet
    df = pd.read_parquet(file_path)

    #Make all the intervetion datasets uniform: make lowercase + add _ as spacing
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=False)
    
    # Filter based on heart disorders
    df = df[df['eventtype_trip'].isin(heart_disorder_events)]
    
    # Further filtering to remove rows with an abandon reason as these people are okay
    df = df[df['abandon_reason'].isna()]
       
    # Drop some columns that are in some datasets but not in all
    if 'province_intervention' in df.columns:
        columns_to_drop.append('province_intervention')

    # drop the rest of the columns 
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Extracting the date that is a string from t0 (least missing values)
    df['date'] = df['t0'].str.extract('(\d{2}[A-Z]{3}\d{2})')[0]
    
    #Extracting the time for t2 till t7 
    t2_till_t7 = ['t2','t3', 't4', 't5', 't6', 't7']

    for col in t2_till_t7:
        # Convert to datetime
        df[col] = pd.to_datetime(df[col], errors='coerce')
        # Extract only the time part and replace the column
        df[col] = df[col].dt.strftime('%H:%M:%S')

    #Fixing t0 and t1 as they have string value in the data
    date_prefix_pattern = r'\d{2}[A-Z]{3}\d{2}:' #this sets the parameters that need to be removerd 01JUN22:

    df['t0'] = df['t0'].str.replace(date_prefix_pattern, '', regex=True)
    df['t1'] = df['t1'].str.replace(date_prefix_pattern, '', regex=True)

    
    # Convert 'date' column to the format dd/mm/yy
    df['date'] = pd.to_datetime(df['date'], format='%d%b%y').dt.strftime('%d/%m/%y')
    
    return df

#Function to count duplicate mission id's 
def count_same_mission(df):
    duplicate_mission_ids = df.duplicated(subset='mission_id', keep=False)
    return duplicate_mission_ids.sum()


### Cleaning Data 

In [36]:
file_path_interventions1 = "Data/Data/interventions1.parquet"
df_internetions_1 = process_interventions_dataset(file_path_interventions1)
file_path_interventions2 = "Data/Data/interventions2.parquet"
df_internetions_2 = process_interventions_dataset(file_path_interventions2)
file_path_interventions3 = "Data/Data/interventions3.parquet"
df_internetions_3 = process_interventions_dataset(file_path_interventions3)

In [37]:
df_internetions_1.head()

,mission_id,latitude_permanence,longitude_permanence,vector_type,latitude_intervention,longitude_intervention,t0,t1,t2,t3,...,t6,t7,intervention_time_(t1confirmed),waiting_time,intervention_duration,departure_time_(t1confirmed),calculated_traveltime_destinatio,calculated_distance_destination,number_of_transported_persons,date
17,10221520015,51.23355,4.49318,Ambulance,51.23266,4.44440,01:26:09,01:28:37,01:32:34,01:34:33,...,02:24:27,02:34:18,6.0,8.0,58.0,4.0,172.0,2031.0,1.0,01/06/22
18,10221520015,51.22249,4.43629,MUG,51.23266,4.44440,01:26:09,01:45:33,01:48:27,01:52:45,...,02:02:47,02:07:44,7.0,26.0,36.0,3.0,NaN,NaN,NaN,01/06/22
22,10221520020,51.18678,5.11457,PIT,51.16376,4.98392,01:51:06,01:53:35,01:59:00,02:09:43,...,02:53:49,03:05:59,15.0,18.0,62.0,5.0,220.0,2119.0,1.0,01/06/22
29,10221520032,51.29857,4.48800,Ambulance,51.29850,4.47723,04:05:01,04:07:22,04:10:38,04:13:43,...,04:28:50,04:28:51,5.0,8.0,23.0,2.0,128.0,1289.0,NaN,01/06/22
30,10221520033,51.22249,4.43629,PIT,51.25461,4.49728,04:06:31,04:09:20,04:12:57,04:23:00,...,06:52:49,05:10:19,14.0,17.0,166.0,3.0,396.0,4414.0,1.0,01/06/22


In [38]:
print(f"Same mission ID in intervention_1 for {count_same_mission(df_internetions_1)} rows")
print(f"Same mission ID in intervention_2 for {count_same_mission(df_internetions_2)} rows")
print(f"Same mission ID in intervention_3 for {count_same_mission(df_internetions_3)} rows")

Same mission ID in intervention_1 for 17632 rows
Same mission ID in intervention_2 for 17681 rows
Same mission ID in intervention_3 for 18254 rows


There are a lot of interventions where a Ambulance and MUG has been sent out to resulting in a dubble registration -> we could combine these but that we have to look at Intervention duration as they are different for Ambulance and Mug. 

In [48]:
def combine_intervention_rows(df):
    # Helper function to combine vector types
    def combine_vector_types(x):
        return " + ".join(sorted(set(x)))

    # Helper function to get the maximum value, assuming the values are numeric or can be converted
    def get_max_value(x):
        def convert_to_float(v):
            try:
                # Try to directly convert to float
                return float(str(v).replace(',', '.'))
            except ValueError:
                # Handle time or duration strings
                parts = v.split(':')
                if len(parts) == 3:
                    # Assuming format is HH:MM:SS.SSS
                    return int(parts[0]) * 3600 + int(parts[1]) * 60 + float(parts[2])
                elif len(parts) == 2:
                    # Assuming format is MM:SS.SSS
                    return int(parts[0]) * 60 + float(parts[1])
                else:
                    # Default case if format is unknown
                    return 0
        max_value = max(x, key=lambda v: convert_to_float(v))
        return max_value if isinstance(max_value, float) else str(max_value)

    # Define aggregation rules
    aggregation_rules = {
        'latitude_permanence': 'first',
        'longitude_permanence': 'first',
        'latitude_intervention': 'first',
        'longitude_intervention': 'first',
        't0': 'first',
        'vector_type': combine_vector_types,
        't1': 'first', 
        'date':'first'
    }

    # Columns for which we need the maximum value
    max_columns = ['t2', 't3', 't4', 't5', 't6', 't7', 'intervention_time_(t1reported)', 
                   'intervention_time_(t1confirmed)', 'waiting_time', 'intervention_duration', 
                   'departure_time_(t1reported)', 'departure_time_(t1confirmed)', 
                   'calculated_traveltime_destination', 'calculated_distance_destination',
                   'calculated_traveltime_destinatio', 'number_of_transported_persons']

    # Add max rules for specific columns
    for col in max_columns:
        if col in df.columns:
            aggregation_rules[col] = get_max_value

    # Group by 'mission_id' and apply the aggregation rules
    df_combined = df.groupby('mission_id', as_index=False).agg(aggregation_rules)
    
    return df_combined


In [49]:
df_internetions_1_comb= combine_intervention_rows(df_internetions_1)
df_internetions_2_comb= combine_intervention_rows(df_internetions_2)
df_internetions_3_comb= combine_intervention_rows(df_internetions_3)
print(f"Same mission ID in intervention_1 for {count_same_mission(df_internetions_1_comb)} rows")
print(f"Same mission ID in intervention_2 for {count_same_mission(df_internetions_2_comb)} rows")
print(f"Same mission ID in intervention_3 for {count_same_mission(df_internetions_3_comb)} rows")

Same mission ID in intervention_1 for 0 rows
Same mission ID in intervention_2 for 0 rows
Same mission ID in intervention_3 for 0 rows


In [ ]:
df_combined_interventions = pd.concat([df_internetions_1_comb, df_internetions_2_comb, df_internetions_3_comb], ignore_index=True)


,mission_id,latitude_permanence,longitude_permanence,latitude_intervention,longitude_intervention,t0,vector_type,t1,t2,t3,...,t6,t7,intervention_time_(t1confirmed),waiting_time,intervention_duration,departure_time_(t1confirmed),calculated_distance_destination,calculated_traveltime_destinatio,date,number_of_transported_persons
0,10221520015,51.233550,4.493180,51.23266,4.44440,01:26:09,Ambulance + MUG,01:28:37,01:48:27,01:52:45,...,02:24:27,02:34:18,7.0,26.0,58.0,4.0,2031.0,172.0,01/06/22,1.0
1,10221520020,51.186780,5.114570,51.16376,4.98392,01:51:06,PIT,01:53:35,01:59:00,02:09:43,...,02:53:49,03:05:59,15.0,18.0,62.0,5.0,2119.0,220.0,01/06/22,1.0
2,10221520032,51.298570,4.488000,51.29850,4.47723,04:05:01,Ambulance,04:07:22,04:10:38,04:13:43,...,04:28:50,04:28:51,5.0,8.0,23.0,2.0,1289.0,128.0,01/06/22,NaN
3,10221520033,51.222490,4.436290,51.25461,4.49728,04:06:31,PIT,04:09:20,04:12:57,04:23:00,...,06:52:49,05:10:19,14.0,17.0,166.0,3.0,4414.0,396.0,01/06/22,1.0
4,10221520042,51.464850,4.472460,51.42894,4.50414,05:38:13,Ambulance + MUG,05:41:30,05:47:02,06:15:39,...,06:47:23,07:21:28,31.0,37.0,69.0,3.0,17279.0,1081.0,01/06/22,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21083,50221910670,50.469758,4.128008,50.49976,4.10360,21:00:26,Ambulance + MUG,21:04:20,21:05:41,21:14:00,...,22:05:03,22:30:42,NaN,14.0,65.0,NaN,NaN,NaN,10/07/22,1.0
21084,50221910739,50.616520,3.375120,50.60429,3.37735,23:53:11,Ambulance + MUG,23:56:07,23:58:32,00:15:06,...,00:47:38,01:00:34,19.0,22.0,54.0,4.0,NaN,NaN,10/07/22,1.0
21085,50221920031,50.446080,3.919160,50.45734,3.94684,02:17:06,Ambulance,02:19:17,02:22:12,02:26:49,...,03:07:53,03:19:40,4.0,9.0,50.0,0.0,NaN,NaN,11/07/22,1.0
21086,50221920035,50.598490,3.383350,50.51713,3.58035,02:41:02,Ambulance + MUG,02:42:44,02:46:18,03:01:23,...,NaN,04:28:47,18.0,20.0,NaN,4.0,NaN,NaN,11/07/22,NaN


In [42]:
file_path_bxl = "Data\Data\interventions_bxl.parquet.gzip"
df_internetions_bxl = process_interventions_dataset(file_path_bxl)


KeyError: "['intervention_time_(t1reported)', 'departure_time_(t1reported)'] not found in axis"

In [ ]:
summarize_dataset("Data\Data\interventions_bxl.parquet.gzip")

,Column,Missing Values,Percentage Missing (%),Data Type
0,unavailable_time,115643,99.996541,float64
1,t9,115592,99.952441,object
2,t1confirmed,115362,99.753560,object
3,housenumber_permanence,113417,98.071718,object
4,abandon_reason,86613,74.894290,object
5,number_of_transported_persons,62159,53.748908,float64
6,housenumber_destination_hospital,62030,53.637362,float64
7,t5,56507,48.861622,object
8,t4,47558,41.123419,object
9,streetname_destination_hospital,47008,40.647833,object


In [ ]:
summarize_dataset("Data\Data\interventions_bxl2.parquet.gzip")


,Column,Missing Values,Percentage Missing (%),Data Type
0,Housenumber destination hospital,38619,99.997411,object
1,Abandon reason FR,34326,88.881409,object
2,Abandon reason NL,34326,88.881409,object
3,Streetname destination hospital,25914,67.099948,object
4,Cityname destination hospital,25914,67.099948,object
5,Name destination hospital,25914,67.099948,object
6,T6,25905,67.076644,object
7,T4,25113,65.025893,object
8,T5,24748,64.080787,object
9,T3,21746,56.307613,object


BXL2 data is very different form BXL1 -> further work needed